In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer


In [3]:
df = pd.read_csv("nan2.csv")

In [5]:
# 1. Manejar valores faltantes en 'peso' con la media
imputer = SimpleImputer(strategy='mean')
df['peso'] = imputer.fit_transform(df[['peso']])

# 2. Eliminar filas con valores faltantes en 'fumador' (si las hubiera)
df.dropna(subset=['fumador'], inplace=True)

In [9]:
# 3. Codificar variables categóricas a numéricas
# Usa los nombres de columna correctos según tu DataFrame
categorical_cols = ['genero', 'nivel_educacional', 'fumador', 'actividad_fisica', 'estado_civil', 'enfermedades_cronicas', 'consumo_alcohol', 'uso_transporte', 'ciudad']
label_encoders = {}
for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

# 4. Eliminar la columna 'id' ya que no es útil para el modelo
if 'id' in df.columns:
    df = df.drop('id', axis=1)

# --- Preparación para el Modelo ---

# Definir variable objetivo y variables predictoras
# En este ejemplo, predecimos 'fumador'
X = df.drop('fumador', axis=1)
y = df['fumador']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Entrenamiento del Modelo ---

# Inicializar y entrenar el clasificador RandomForest
# n_estimators: número de árboles en el bosque
# random_state: para reproducibilidad
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# --- Evaluación del Modelo ---

# Realizar predicciones en el conjunto de prueba
y_pred = rf_classifier.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.4f}")

# Mostrar un reporte de clasificación más detallado
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar la importancia de cada característica
print("\nImportancia de las Características:")
feature_importances = pd.DataFrame(rf_classifier.feature_importances_,
                                   index = X_train.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

Precisión del modelo: 0.4651

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.46      0.54      0.50       180
           1       0.48      0.39      0.43       192

    accuracy                           0.47       372
   macro avg       0.47      0.47      0.46       372
weighted avg       0.47      0.47      0.46       372


Importancia de las Características:
                       importance
ingresos                 0.160721
edad                     0.154767
peso                     0.149569
horas_suenho             0.138798
ciudad                   0.076193
estado_civil             0.059319
nivel_educacional        0.058359
uso_transporte           0.048660
actividad_fisica         0.047543
consumo_alcohol          0.043995
genero                   0.033864
enfermedades_cronicas    0.028212
